![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Ports

## What is a `Port`?!

A `Port` is a collection of variables that helps transfer data from one `System` to another.
Without such an object, connections between systems should be done variable-by-variable, *outch!*

![Connected ports](images/ports_1.svg)

## Types

### `IN` or `OUT`

Port instances inside systems are directional objects; they can be either `IN` or `OUT`. This definition ensures compatibility at connection.
However, the definition of a `Port` class only contains the list and description of its variables, irrespective of direction.

### Available ports and compatibility

<font color='orange'>**CoSApp**</font> allow user-defined ports, stored in libraries or created in specific projects. They define a frozen collection of variables that can be shared between multiple systems. They are compatible by construction (see [Port connection](#Port-connection) section of this tutorial).

## Create a port

### Import CoSApp core package

In [ ]:
# import cosapp base classes
from cosapp.base import System, Port

#### Define a new port

In [ ]:
class DemoPort(Port):
    
    def setup(self):
        self.add_variable('a', 1.0)
        self.add_variable('b', 2.0)
        self.add_variable('c', 3.0)

#### Use it in a system

We now create a new `System` using `DemoPort`:

In [ ]:
class DemoSystem(System):

    def setup(self):
        self.add_input(DemoPort, 'p_in')
        self.add_output(DemoPort, 'p_out') 

    def compute(self):
        self.p_out.a = self.p_in.c
        self.p_out.b = self.p_in.b
        self.p_out.c = self.p_in.a

s = DemoSystem(name='s')

In the example above, `p_in` and `p_out` are two instances of `DemoPort`.
All instances of `DemoSystem`, such as `s`, will possess attributes `p_in` and `p_out`.
Use of methods `add_input` and `add_output` provides a clear interface to create ports with the desired direction.

![Port-DemoSystem](images/ports_2.svg)

Run the system to confirm the expected behaviour

In [ ]:
s.run_once()
print('s.p_in:')
s.p_in

In [ ]:
print('s.p_out:')
s.p_out

#### Set information on the variables

All `Port` variables may be given optional information:

- `unit`: Physical unit of the variable, given by a string. Units *are not enforced* inside a `System`.
This means that system developers are responsible for unit conversions (if required) in method `compute`. However, **CoSApp** will take care of unit conversion during data transfer via a connector (see [Port connection](#Port-connection)).
- `desc`: Short description of the variable.
- `dtype`: If you need to force certain data type(s) on a variable, a tuple of acceptable types can be provided
through this keyword. If that information is not supplied, dtype is inferred from the variable value; e.g.
a number (integer or floating point) will be typed as `Number`.

In [ ]:
class AdvancedDemoPort(Port):
    
    def setup(self):
        self.add_variable('a', 3e2, unit='degK', dtype=float, desc='Temperature')
        self.add_variable('b', 0.1, unit='MPa', dtype=(int, float), desc='Pressure')
        self.add_variable('c', 1.0, unit='kg/s', desc='Mass flowrate')


class AdvancedDemoSystem(System):

    def setup(self):
        self.add_input(AdvancedDemoPort, 'p_in')
        self.add_output(AdvancedDemoPort, 'p_out') 

    def compute(self):
        self.p_out.a = self.p_in.c
        self.p_out.b = self.p_in.b
        self.p_out.c = self.p_in.a


sa = AdvancedDemoSystem('sa')
print('Output port')
sa.p_in

## Port connection

### Introduction

Port connection creates a dedicated object (of type `Connector`) managing one-way data transfers between two `Port` instances.
Connectors are handled at system levels, via method `connect`, and can be accessed through attribute `connectors` as a dictionnary.

![Port-Connection](images/ports_3.svg)


In [ ]:
h = System('head')
h.add_child(DemoSystem('demo1'))
h.add_child(DemoSystem('demo2'))

h.connect(h.demo2.p_in, h.demo1.p_out)

If you need to connect a subset of the `Port` variables, or connect inwards or outwards, the `connect()` method of `System` can be called in `setup` with additional arguments, providing variable mapping from the first port to the other.

In [ ]:
class MonitorSystem(System):

    def setup(self):
        self.add_inward('a')
        self.add_inward('b')
        self.add_inward('x')
        self.add_outward('result')

    def compute(self):
        self.result = self.a

h.add_child(MonitorSystem('monitor'))
# Connect `h.monitor.a` to `h.demo1.p_out.a`
h.connect(h.monitor.inwards, h.demo1.p_out, 'a')

h.demo1.p_in.a = 50.
h.demo1.p_in.b = 0.
h.demo1.p_in.c = 25.
h.run_once()

print(f"h.monitor.a = {h.monitor.a}")
h.demo1.p_out

In the example above, variable `a` from port `inwards` has been connected to variable `a` from port `demo1.p_out`.
This is the simplest mapping, when the variable name is identical in poth ports.

Other options are possible:

- A list variable names, if all names exist in both ports:
```python
    h.connect(h.monitor.inwards, h.demo1.p_out, ['a', 'b'])
```

- An explicit name mapping, if names differ:
```python
    h.connect(h.monitor.inwards, h.demo1.p_out, {'a': 'a', 'x': 'c'})
```

- Connecting two systems, with a port and/or variable mapping:
```python
    h.connect(h.monitor, h.demo1, {'x': 'p_out.c'})
```
In the last example, `h.x` will be connected to `h.demo1.p_out.c`.
Note that `h.x` and `h.inwards.x` refer to the same variable.

In [ ]:
print('h.demo2.p_in')
h.demo2.p_in

In [ ]:
print('h.monitor.outwards')
h.monitor.outwards

**Congrats!** Now you have the basic of `Port` in **CoSApp**

Next you will discover how to solve mathematical problem using [Drivers](03-Drivers.ipynb).